In [ ]:
import numpy as np
import scipy as sp
import cvxpy as cvx

N = 10
k = 100
d = 400


In [ ]:
A = np.random.randn(k, d, N)

for s in range(1, d + 1):
    X = np.random.randn(s, N)
    for n in range(N):
        S = np.random.choice(d, s, replace = True)
        x = np.zeros(d, dtype="float64")
        x[S] = X[:, n]
        y = np.dot(A[:, :, n], x)
        
        z = cvx.Variable(d)

In [ ]:
s = 5
S = np.random.choice(d, s, replace = True)
X = np.random.randn(s, N)
A = np.random.randn(k, d, N)

In [ ]:
n = 0
x = np.zeros(d, dtype="float64")
x[S] = X[:, n]


In [ ]:
y = np.dot(A[:, :, n], x)
z = cvx.Variable(d)
constraints = [A[:, :, n] * z == y,]
objective = cvx.Minimize(cvx.norm(z, 1))
prob = cvx.Problem(objective, constraints)
prob.solve()

In [ ]:
np.where(abs(z.value) > 1e-10), S
